In [2]:
import numpy as np
import pandas as pd
import plotly
import plotly.graph_objs as go
import plotly.express as px
from plotly.subplots import make_subplots

# Preparing data for visualization 
- fact (total sales) (sample)
- channel (all)
- market (all)
- market_product (sample)
- product (sample(?))

In [3]:
df_fact = pd.read_excel('test_task_full.xlsx', 'fact')

In [4]:
# cut off NULLs and transform data to int 
df_fact = df_fact[df_fact['product_key'] != '[NULL]'].astype(int)

In [5]:
# tranform date_key to month only
df_fact['date_key'] = df_fact['date_key']%10000//100

In [6]:
df_channel = pd.read_excel('test_task_full.xlsx', 'channel')

In [7]:
df_market = pd.read_excel('test_task_full.xlsx', 'market')

In [14]:
df_market_all = pd.read_excel('test_task_full.xlsx', 'market')

In [8]:
df_market_product = pd.read_excel('test_task_full.xlsx', 'market_product')

In [9]:
df_product = pd.read_excel('test_task_full.xlsx', 'product')

In [58]:
def bar_chart(df, title):
    fig = go.Figure(go.Bar(x = df.index, y = df.values, text = df.values, textposition='auto'))
    fig.update_layout(title=title)
    fig.show()

In [61]:
def pie_chart(df, title):
    fig = go.Figure()
    fig.add_trace(go.Pie(values=df, labels=df.index))
    fig.update_layout(title=title)
    fig.show()

In [209]:
def filtered_market_bar(df, market, title, top=5):
    filtered_market = df[df['market_name'] == market].groupby('eph_mra_3')['product_key'].count().sort_values(ascending=False)
    bar_chart(filtered_market.head(top), title)

In [213]:
def filtered_sales_bar(df, market, title, currency, top=5):
    filtered_market = df[df['market_name'] == market].groupby('eph_mra_3')[currency].sum().sort_values(ascending=False)
    bar_chart(filtered_market.head(top), title)

In [254]:
def top_products_sales_bar(df, emp_mra, title, currency, top=10):
    filtered_market = df[df['eph_mra_3'] == emp_mra].groupby('trade_name')[currency].sum().sort_values(ascending=False)
    bar_chart(filtered_market.head(top), title)

# Visual market information

In [17]:
# cut off Totals
for el in df_market['market_name']: 
    if el[-5:] == 'Total':
        df_market.drop(df_market[df_market['market_name'] == el].index, inplace=True)

In [141]:
# join tables product on market
df_product_on_market = df_product.merge(df_market_product.merge(
    df_market, how='left', on='market_key'), how='left', on='product_key')

## Market Segmentation by Products

In [137]:
# sorted markets by product
markets_by_products = df_product_on_market.groupby('market_name')['product_key'].count()

In [182]:
pie_chart(markets_by_products, 'Market Segmentation by Products')

### Top-5 Markets by Products

In [57]:
bar_chart(markets_by_products.head(5), 'Top-5 Markets by Products')

## Top Categories EPH MRA 3 on Markets by Products 

In [202]:
# cut off eph_mra NULLs
df_markets_clean = df_product_on_market[df_product_on_market['eph_mra_3'] != '[NULL]']

### Oncology Chemotherapy

In [196]:
filtered_market_bar(df_markets_clean,'Oncology_Chemotherapy', 'Top-10 Categories in Oncology Chemotherapy', 10)

### CMC Cardio BB  

In [197]:
filtered_market_bar(df_markets_clean,'CMC_Cardio_BB', 'Top-3 Categories in CMC Cardio BB', 3)

### CMC Cardio BB mono  

In [198]:
filtered_market_bar(df_markets_clean,'CMC_Cardio_BB_mono', 'Top-3 Categories in CMC Cardio BB mono', 3)

### CMC Nootropics 

In [199]:
filtered_market_bar(df_markets_clean,'CMC_Nootropics', 'Top-3 Categories in CMC Nootropics', 3)

### Oncology H&N 

In [200]:
filtered_market_bar(df_markets_clean,'Oncology_H&N', 'Top-5 Categories in Oncology H&N')

## Market Segmentation by Sales in eur

In [184]:
# join tables fact on product on market sales)
df_sales = df_fact.merge(df_product_on_market, how='left', on='product_key')__
market_by_sales = df_sales.groupby('market_name')['trd_eur'].sum()

In [181]:
pie_chart(market_by_sales, 'Market Segmentation by Sales')

In [225]:
bar_chart(market_by_sales.sort_values(ascending=False).head(5), 'Top-5 Markets by Sales')

## Top Categories EPH MRA 3 on Markets by Products

In [201]:
# cut off eph_mra NULLs
df_sales_clean = df_sales[df_sales['eph_mra_3'] != '[NULL]']

### CMC Cardio BB

In [228]:
filtered_sales_bar(df_sales_clean, 'CMC_Cardio_BB', 'Top Categories in CMC Cardio BB by sales in eur', 'trd_eur')

### CMC Cardio BB mono

In [231]:
filtered_sales_bar(df_sales_clean, 'CMC_Cardio_BB_mono', 'CMC Cardio BB mono by sales in eur', 'trd_eur')

### Oncology_TRG

In [234]:
filtered_sales_bar(df_sales_clean, 'Oncology_TRG', 'Top Categories in Oncology TRG by sales in eur', 'trd_eur')

### CMC_Cardio_Bisoprolol_mono

In [233]:
filtered_sales_bar(df_sales_clean, 'CMC_Cardio_Bisoprolol_mono', 'Top Categories in CMC Cardio Bisoprolol mono by sales in eur', 'trd_eur')

### CMC OAD SGLT2

In [229]:
filtered_sales_bar(df_sales_clean, 'CMC_OAD_SGLT2', 'Top Categories in CMC OAD SGLT2 by sales in eur', 'trd_eur')

## Top-10 Products in Top Categories by sales

In [240]:
df_sales.groupby('market_name')['trd_eur'].sum().sort_values(ascending=False).head(5)

market_name
CMC_Cardio_BB                 946106
CMC_Cardio_BB_mono            869524
Oncology_TRG                  824772
CMC_Cardio_Bisoprolol_mono    651845
CMC_OAD_SGLT2                 523043
Name: trd_eur, dtype: int64

In [244]:
top_categories = df_sales_clean.groupby('eph_mra_3')['trd_eur'].sum().sort_values(ascending=False).head(5)
top_categories

eph_mra_3
C07A - BETA BLOCKING AGENT PLAIN    4444410
A10P - SGLT2 INHIBITOR A-DIABS      2092172
L01H - PROTEIN KINASE INH A-NEO     1725522
A16A - OTHER METABOLIC PRODUCTS     1300001
M03A - MUSCLE RELAXANTS,PERIPHER    1086536
Name: trd_eur, dtype: int64

In [250]:
df_sales_clean[df_sales_clean['eph_mra_3'] == 'C07A - BETA BLOCKING AGENT PLAIN'].groupby('trade_name')['trd_eur'].sum()

trade_name
ANAPRILIN                7568
ATENOLOL                  368
ATENOLOL NYCOMED            0
ATENOLOL-TEVA            1623
BETALOC                   184
BETALOC ZOK            312596
BISOPROLOL             132185
CARVEDILOL-AKRIKHI      21344
CONCOR                3127040
EGILOK                  74768
LOKREN                  41784
METOPROLOL              43298
METOPROLOL-AKRIKHI      28740
METOPROLOL-TEVA          1128
NEBILET                 85288
REVELOL XL 100           1936
REVELOL XL 50           10124
SOTAHEXAL              550444
VEDICARDOL               3992
Name: trd_eur, dtype: int64

In [ ]:
BETA BLOCKING AGENT PLAIN

In [255]:
for category in top_categories.index:
    top_products_sales_bar(df_sales_clean, category, 'Test', 'trd_eur')

In [ ]:
SGLT2 INHIBITOR A-DIABS

In [ ]:
PROTEIN KINASE INH A-NEO

In [ ]:
OTHER METABOLIC PRODUCTS

In [ ]:
MUSCLE RELAXANTS,PERIPHER